## Compute source time course (STC) from Raw and Epochs data object and save as pickle for later analyses
#### Input: *_eyes_open-raw.fif, *_eyes_closed-raw.fif, *-epo.fif
####  Output: *_label_ts.pkl

In [ ]:
import os
from pathlib import Path
import sys
sys.path.append('/home/wanglab/Documents/George Kenefati/Code/eeg_toolkit/')
from eeg_toolkit import utils, preprocess, source_localization

### Paths

In [ ]:
# Time window for source localization computation
times_tup,time_win_path = preprocess.get_time_window(5)

# Average dipoles, always true unless you have subject-specific MRI and know what you're doing
average_dipoles = True

# Save inverse. True if you want to plot the brain model with AEC connectivity
save_inv = True

In [ ]:
# Read in processed raw data
data_dir = Path("../../Data")
processed_data_path = data_dir / 'Processed Data'
epo_path = data_dir / time_win_path

### Subject IDs

In [ ]:
# viable subjects
sub_ids = utils.import_subs(os.path.join(data_dir),'sub_ids.txt')
# select only 64ch subs and turn into set
sub_ids = sub_ids[7:]
print(sub_ids)

In [ ]:
print(f"Chronics: {len([el for el in sub_ids if el.startswith('0')])}")
print(f"Controls: {len([el for el in sub_ids if el.startswith('C')])}")
print(f"Total: {len(sub_ids)}")

#### Look for subjects who do not have EO or EC cropped data

In [ ]:
# For edge cases of subjects missing eyes open or eyes closed data
no_eyes_open = []
no_eyes_closed = []

for sub_id in sub_ids:
    if not os.path.exists(os.path.join(processed_data_path, f"{sub_id}_eyes_closed-raw.fif")):
        no_eyes_closed.append(sub_id)
        print(f"Subject: {sub_id} missing eyes closed data")
    if not os.path.exists(os.path.join(processed_data_path, f"{sub_id}_eyes_open-raw.fif")):
        no_eyes_open.append(sub_id)
        print(f"Subject: {sub_id} missing eyes open data")
        
print(no_eyes_open)
print(no_eyes_closed)

In [ ]:
# Get stc only from selected labels
roi_names = [# Left
             'rostralanteriorcingulate-lh', # Left Rostral ACC
             'caudalanteriorcingulate-lh', # Left Caudal ACC
             'postcentral-lh', # Left S1,
             'insula-lh', 'superiorfrontal-lh', # Left Insula, Left DL-PFC,
             'medialorbitofrontal-lh', # Left Medial-OFC
             # Right
             'rostralanteriorcingulate-rh', # Right Rostral ACC
             'caudalanteriorcingulate-rh', # Right Caudal ACC
             'postcentral-rh', # , Right S1
             'insula-rh', 'superiorfrontal-rh', # Right Insula, Right DL-PFC
             'medialorbitofrontal-rh', # Right Medial-OFC
]


### Compute STCs

In [ ]:
nan_subjects=[]

methods = [
    'MNE',
    'dSPM',
    ]

for sub_id in sub_ids: 
    # Check if eyes open or eyes closed need to be computed
    EO_bool = False if sub_id in no_eyes_open else True
    if not EO_bool:
        print(f"Subject: {sub_id} missing eyes open data")
        break
    
    for save_stc_mat in [
        False,
        True,
        ]:    
        for method in methods:
            print(f"Source Localization Method: {method}")                          

            # Paths
            stc_path = (data_dir / f'Source Time Courses ({method}) (MAT)' 
                        if save_stc_mat 
                        else data_dir / f'Source Time Courses ({method})')
                           
            EO_resting_save_path = stc_path / "Eyes Open"
            EC_resting_save_path = stc_path / "Eyes Closed"
            zscored_epochs_save_path = stc_path / "zscored_Epochs" / time_win_path
            save_paths = [EC_resting_save_path,EO_resting_save_path,zscored_epochs_save_path] 
            [os.makedirs(path) for path in save_paths if not os.path.exists(path)]
            
            # No resting if exporting to MAT
            if save_stc_mat:
                Epochs_bool = True
                EC_bool = False
                EO_bool = False
            else:
                Epochs_bool = True

            # Compute source localization for subject and save 
            label_ts_All_Conds, sub_id_if_nan = source_localization.to_source(sub_id,
                                                                              processed_data_path,
                                                                              zscored_epochs_save_path,
                                                                              EC_resting_save_path,
                                                                              EO_resting_save_path,
                                                                              roi_names,
                                                                              times_tup,
                                                                              method=method,
                                                                              return_zepochs= Epochs_bool,
                                                                              return_EC_resting = False,
                                                                              return_EO_resting = EO_bool,
                                                                              average_dipoles=average_dipoles,
                                                                              save_stc_mat=save_stc_mat,
                                                                              save_inv=save_inv,
                                                                              )
            utils.clear_display()

### Assess results

In [ ]:
nan_subjects

In [ ]:
label_ts_All_Conds